## 1. Preámbulo

Extraemos los datos que necesitamos de los CSV ya cocientados. Necesitamos tener los archivos `cociente_clean_uniform_oX.csv`

In [ ]:
from pyspark import SparkContext
sc=SparkContext()

In [ ]:
rdd=sc.textFile('cociente_clean_uniform_od.csv').map(lambda x: x.split(';'))
rdd2=sc.textFile('cociente_clean_uniform_oi.csv').map(lambda x: x.split(';'))
print(rdd.take(20))

###  [Definición de la distancia de edición para comparar cadenas de texto]

Función recursiva que devuelve el número de ediciones que separan dos cadenas de texto. No se utiliza por ser muy pesada su evaluación.


In [ ]:
def iterative_levenshtein(s, t):
    rows = len(s)+1
    cols = len(t)+1
    dist = [[0 for x in range(cols)] for x in range(rows)]
    # source prefixes can be transformed into empty strings 
    # by deletions:
    for i in range(1, rows):
        dist[i][0] = i
    # target prefixes can be created from an empty source string
    # by inserting the characters
    for i in range(1, cols):
        dist[0][i] = i
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = 1
            dist[row][col] = min(dist[row-1][col] + 1,      # deletion
                                 dist[row][col-1] + 1,      # insertion
                                 dist[row-1][col-1] + cost) # substitution
    
 
    return dist

In [ ]:
def LD(s, t, cum = 0,it=0):
    print("eval",it)
    if cum >3:
        return 99
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
       
    res = min([LD(s[:-1], t, cum+1,it+1)+1,
               LD(s, t[:-1], cum+1,it+1)+1, 
               LD(s[:-1], t[:-1], cum+cost,it+1) + cost])
    return res

In [ ]:
def levenshtein(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    cost = 0 if s[-1] == t[-1] else 1
       
    i1 = (s[:-1], t)
    if not i1 in memo:
        memo[i1] = levenshtein(*i1)
    i2 = (s, t[:-1])
    if not i2 in memo:
        memo[i2] = levenshtein(*i2)
    i3 = (s[:-1], t[:-1])
    if not i3 in memo:
        memo[i3] = levenshtein(*i3)
    res = min([memo[i1]+1, memo[i2]+1, memo[i3]+cost])
    return res

## 2. Definición de la función de Matching

Se utiliza como condición que coincidan 3 campos como approach básico. La función devuelve un número con el tipo de error si lo encuentra, y en caso contrario devuelve `False`. La correspondencia es:
1. Error en el número de historia
2. Error en los apellidos
3. Error en el nombre
4. Error en la fecha de nacimiento

In [ ]:
def Matching(x,y):

    if (x[2]==y[2] and x[1]==y[1] and x[3]==y[3]):
        return 0
    elif x[2]==y[2] and x[0]==y[0] and x[3]==y[3]:
        return 1
    elif x[0]==y[0] and x[1]==y[1] and x[3]==y[3]:
        return 2
    elif x[2]==y[2] and x[1]==y[1] and x[0]==y[0]:
        return 3
    else:
        return False

Intento de optimización reduciendo el numero de comparaciones mediante anidados de condicionales.

In [ ]:
def matching_simple(x,y):
    if x[0]==y[0]:
        if x[1]==y[1]:
            if x[3]==y[3]:
                return 2   #Error en el campo 3
            elif x[2]==y[2]:
                return 3   #Error en el campo 2
        elif x[2]==y[2] and x[3]==y[3]:
            return 1       #Error en el campo 1
    elif x[1]==y[1] and x[2]==y[2] and x[3]==y[3]:
        return 0           #Error en el campo 0
    else:
        return False

In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
def match_complejo(x,y):
    if x[0]==y[0]:
        if x[3]==y[3]:
            if x[2]==y[2] or fuzz.ratio(x[2],y[2])>90:
                return 1
            if x[1]==y[1] or fuzz.ratio(x[1],y[1])>90:
                return 2
        elif fuzz.ratio(x[2],y[2])>90 and fuzz.ratio(x[1],y[1])>90:
            return 3
    elif x[3]==y[3]:
        if fuzz.ratio(x[2],y[2])>90:
            if fuzz.ratio(x[1],y[1])>90:
                return 0
    else:
        return False

## 3. Ejecución y conteo de errores

In [ ]:
import time

### 3.1 Prueba usando match_complejo

Tarda aproximadamente 250 segundos en los ordenadores de la facultad.

In [ ]:
Error=[0,0,0,0]   # Vector que contendrá el número de errores separados por tipo 
t=time.time()
pruebas=rdd.collect()     # Se hace un collect para poder comparar todos con todos
revisados=set()
for x in range(len(pruebas)):
    for y in range(x+1,len(pruebas)):
        if not (x in revisados and y in revisados):
            M=match_complejo(pruebas[x],pruebas[y])
            if type(M)==int:       # Se comprueba si se ha hay matching
                Error[M]+=1
                revisados.add(y)
            
print("Tiempo: ", time.time()-t)
print("Errores: ", Error)
print("Suma: ", sum(Error))

In [ ]:
### OJO IZQUIERDO!! ###
Error=[0,0,0,0]   # Vector que contendrá el número de errores separados por tipo 
t=time.time()
pruebas=rdd2.collect()     # Se hace un collect para poder comparar todos con todos
for x in range(len(pruebas)):
    for y in range(x+1,len(pruebas)):
        if not (x in revisados and y in revisados):
            M=match_complejo(pruebas[x],pruebas[y])
            if type(M)==int:       # Se comprueba si se ha hay matching
                Error[M]+=1
                revisados.add(y)
                
print("Tiempo: ", time.time()-t)
print("Errores: ", Error)
print("Suma: ", sum(Error))

### 3.2 Prueba usando matching_simple

Tarda aproximadamente 200 segundos en los ordenadores de la facultad.

In [ ]:
Error=[0,0,0,0]
t=time.time()
pruebas=rdd.collect()
revisados=set()
for x in range(len(pruebas)):
    for y in range(x+1,len(pruebas)):
        if not (x in revisados and y in revisados):
            M=matching_simple(pruebas[x],pruebas[y])
            if type(M)==int:
                revisados.add(y)
                Error[M]+=1
            
print("Tiempo: ", time.time()-t)
print("Errores: ", Error)
print("SUMA: ", sum(Error))

### Ojo izquierdo

In [ ]:
Error=[0,0,0,0]
t=time.time()
pruebas=rdd2.collect()
revisados=set()
for x in range(len(pruebas)):
    for y in range(x+1,len(pruebas)):
        if not (x in revisados and y in revisados):
            M=matching_simple(pruebas[x],pruebas[y])
            if type(M)==int:
                revisados.add(y)
                Error[M]+=1
            
print("Tiempo: ", time.time()-t)
print("Errores: ", Error)
print("SUMA: ", sum(Error))

### raw

In [ ]:
rdd3=sc.textFile('cociente_raw_uniform_od.csv').map(lambda x: x.split(';'))
rdd4=sc.textFile('cociente_raw_uniform_oi.csv').map(lambda x: x.split(';'))
print(rdd.take(20))

In [ ]:
print(rdd3.count())
print(rdd4.count())

In [ ]:
Error=[0,0,0,0]
t=time.time()
pruebas=rdd3.collect()
for x in range(len(pruebas)):
    for y in range(x+1,len(pruebas)):
        M=matching_simple(pruebas[x],pruebas[y])
        if type(M)==int:
            Error[M]+=1
            
print("Tiempo: ", time.time()-t)
print("Errores: ", Error)

In [ ]:
Error=[0,0,0,0]
t=time.time()
pruebas=rdd4.collect()
for x in range(len(pruebas)):
    for y in range(x+1,len(pruebas)):
        M=matching_simple(pruebas[x],pruebas[y])
        if type(M)==int:
            Error[M]+=1
            
print("Tiempo: ", time.time()-t)
print("Errores: ", Error)

## 4. Registro de los errores

Guardamos un archivo llamado `errores_oX.csv` con los errores en cada ojo de la forma: `Tipo error, Registro 1, Registro 2`. 

In [ ]:
with open('errores_od.csv', 'w') as archivo:
    pruebas=rdd.collect()
    for x in range(len(pruebas)):
        for y in range(x+1,len(pruebas)):
            M=matching_simple(pruebas[x],pruebas[y])
            if type(M)==int:
                print(';'.join([str(M)]+pruebas[x]+pruebas[y]), file=archivo, end='\n') 

In [ ]:
with open('errores_oi.csv', 'w') as archivo:
    pruebas=rdd2.collect()
    for x in range(len(pruebas)):
        for y in range(x+1,len(pruebas)):
            M=matching_simple(pruebas[x],pruebas[y])
            if type(M)==int:
                print(';'.join([str(M)]+pruebas[x]+pruebas[y]), file=archivo, end='\n') 

In [ ]:
type(False)